In [ ]:
class Token:
  def __init__(self,id):
    self.id = id

In [ ]:
class Task_Token(Token):
  def __init__(self,id,name,DOR):
    super().__init__(id)
    self.name = name
    self.DOR = DOR

In [ ]:
class DOR_Token(Token):
  def __init__(self,id,name):
    super().__init__(id)
    self.campos = list(name)

In [ ]:
class startEvent_Token(Token):
  def __init__(self,id,name):
    super().__init__(id)
    self.name = name

In [ ]:
class sequenceFlow_Token(Token):
  def __init__(self,id,entrada,salida,name=None):
    super().__init__(id)
    self.entrada = entrada
    self.salida = salida
    self.name=name

In [ ]:
class endEvent_Token(Token):
  def __init__(self,id,name=None):
    super().__init__(id)
    self.name = name

In [ ]:
class gateway_Token(Token):
  def __init__(self,id,name):
    super().__init__(id)
    self.name = name

In [ ]:
#En el parse poner el nombre del archivo dentro de los paréntesis

import xml.dom.minidom
domtree = xml.dom.minidom.parse('bpmn_tienda.bpmn')
definitions = domtree.documentElement
#Función para obtener todos los elementos/Nodos hijo de los procesos
def getElement(nodeList):
  rc = []
  for node in nodeList:
    if node.nodeType == node.ELEMENT_NODE:
      rc.append(node)
  return rc

In [ ]:
colab = definitions.getElementsByTagName('bpmn:collaboration')

participantes = []
for e in colab:
  p = e.childNodes
  participantes.append(getElement(p))
participantes

[[<DOM Element: bpmn:participant at 0x7f6eef1799c8>,
  <DOM Element: bpmn:participant at 0x7f6eef179a60>]]

In [ ]:
#Definir los procesos dentro del XML
processes = definitions.getElementsByTagName('bpmn:process')
for process in processes:
    print("---Process---")
    print(process)
    print(process.childNodes)
    if process.hasAttribute('id'):
        print("ID: {}".format(process.getAttribute('id')))

---Process---
<DOM Element: bpmn:process at 0x7f6eef179af8>
[<DOM Text node "'\n    '">, <DOM Element: bpmn:task at 0x7f6eef179b90>, <DOM Text node "'\n    '">, <DOM Element: bpmn:startEvent at 0x7f6eef179d58>, <DOM Text node "'\n    '">, <DOM Element: bpmn:task at 0x7f6eef179e88>, <DOM Text node "'\n    '">, <DOM Element: bpmn:exclusiveGateway at 0x7f6eee8580e0>, <DOM Text node "'\n    '">, <DOM Element: bpmn:task at 0x7f6eee8583d8>, <DOM Text node "'\n    '">, <DOM Element: bpmn:task at 0x7f6eee8585a0>, <DOM Text node "'\n    '">, <DOM Element: bpmn:dataObjectReference at 0x7f6eee8586d0>, <DOM Text node "'\n    '">, <DOM Element: bpmn:dataObject at 0x7f6eee858768>, <DOM Text node "'\n    '">, <DOM Element: bpmn:task at 0x7f6eee858800>, <DOM Text node "'\n    '">, <DOM Element: bpmn:task at 0x7f6eee858a60>, <DOM Text node "'\n    '">, <DOM Element: bpmn:task at 0x7f6eee858b90>, <DOM Text node "'\n    '">, <DOM Element: bpmn:dataObjectReference at 0x7f6eee858df0>, <DOM Text node "'\n  

In [ ]:
#Lista que contiene los elementos de cada proceso en sublistas
EleNodes = []
for process in processes:
  nodes = process.childNodes
  EleNodes.append(getElement(nodes))

In [ ]:
#hace listas de objetos de cada tipo de token
tasks=[]
starts=[]
DORs=[]
flows=[]
ends=[]
gateways=[]
for node in EleNodes:
  for element in node:
    if 'task' in element.tagName:
      if element.childNodes:
        hijos = getElement(element.childNodes)
        for hijo in hijos:
         if 'dataOutputAssociation' in hijo.tagName:
          nodes = getElement(hijo.childNodes)
          for node in nodes:
            if 'targetRef' in node.tagName:
              DOR = node.firstChild.nodeValue
              tasks.append(Task_Token(element.getAttribute('id'),element.getAttribute('name'),DOR))
      else:
        tasks.append(Task_Token(element.getAttribute('id'),element.getAttribute('name')))
    elif 'startEvent' in element.tagName:
      starts.append(startEvent_Token(element.getAttribute('id'),element.getAttribute('name')))
    elif 'dataObjectReference' in element.tagName:
      DORs.append(DOR_Token(element.getAttribute('id'),element.getAttribute('name').split('\n')))
    elif 'sequenceFlow' in element.tagName:
      entrada = element.getAttribute('sourceRef')
      salida = element.getAttribute('targetRef')
      if element.hasAttribute('name'):
        flows.append(sequenceFlow_Token(element.getAttribute('id'),entrada,salida,element.getAttribute('name')))
      else:
        flows.append(sequenceFlow_Token(element.getAttribute('id'),entrada,salida))
    elif 'endEvent' in element.tagName:
      if element.hasAttribute('name'):
        ends.append(endEvent_Token(element.getAttribute('id'),element.getAttribute('name')))
      else:
        ends.append(endEvent_Token(element.getAttribute('id')))
    elif 'exclusiveGateway' in element.tagName:
      gateways.append(gateway_Token(element.getAttribute('id'),element.getAttribute('name')))

In [ ]:
def conectar_salida(lista):
  for el in lista:
    el.salida = []
    for f in flows:
      if el.id == f.entrada:
        el.salida.append(f)
        f.entrada = el
    if not el.salida:
      el.salida = None

In [ ]:
conectar_salida(tasks)
conectar_salida(starts)
conectar_salida(gateways)

In [ ]:
def conectar_entrada(lista):
  for f in flows:
    for el in lista:
      if f.salida == el.id:
        f.salida = el
        el.entrada = f

In [ ]:
conectar_entrada(tasks)
conectar_entrada(DORs)
conectar_entrada(ends)
conectar_entrada(gateways)

In [ ]:
for dor in DORs:
  print(dor.campos)

['integer codigo', 'string nombre', 'integer precio', 'string categoria']
['integer codigo', 'string comprador', 'string vendedor', 'string descripcion', 'integer monto']
['string nombre', 'string usuario', 'string contraseña']
['integer codigo', 'string comprador', 'string vendedor', 'string descripcion', 'integer monto']
['integer id', 'string nombre', 'string correo', 'string tipo_usuario', 'integer telefono']
['integer codigo', 'string comprador', 'string vendedor', 'string descripcion', 'integer monto']


In [ ]:
class atributo:
  def __init__(self,nombre,tipo):
    self.nombre = nombre
    self.tipo = tipo

In [ ]:
class Dato:
  def __init__(self,nombre,atributos):
    self.nombre = nombre
    self.atributos = atributos
Datos = []

In [ ]:
for task in tasks:
  if task.DOR:
    for DOR in DORs:
      if DOR.id == task.DOR:
        task.DOR = DOR
        atributos = []
        for campo in DOR.campos:
          attrs = campo.split(" ")
          atributos.append(atributo(attrs[1], attrs[0]))
        Datos.append(Dato(task.name,atributos))
json = []
for dato in Datos:
  dic = {}
  #print("nombre", dato.nombre)
  att = []
  for attr in dato.atributos:
    dic1 = {}
    #print(" atributo: nombre:", attr.nombre, "tipo: ", attr.tipo)
    dic1.setdefault('name', attr.nombre)
    dic1.setdefault('attribute_type', attr.tipo)
    att.append(dic1)
  dic.setdefault('name', dato.nombre)
  dic.setdefault('attributes', att)
  json.append(dic)
json

[{'attributes': [{'attribute_type': 'integer', 'name': 'codigo'},
   {'attribute_type': 'string', 'name': 'comprador'},
   {'attribute_type': 'string', 'name': 'vendedor'},
   {'attribute_type': 'string', 'name': 'descripcion'},
   {'attribute_type': 'integer', 'name': 'monto'}],
  'name': 'VER pedidos'},
 {'attributes': [{'attribute_type': 'string', 'name': 'nombre'},
   {'attribute_type': 'string', 'name': 'usuario'},
   {'attribute_type': 'string', 'name': 'contraseña'}],
  'name': 'MODIFICAR datos de cuenta'},
 {'attributes': [{'attribute_type': 'integer', 'name': 'codigo'},
   {'attribute_type': 'string', 'name': 'nombre'},
   {'attribute_type': 'integer', 'name': 'precio'},
   {'attribute_type': 'string', 'name': 'categoria'}],
  'name': 'CREAR productos'},
 {'attributes': [{'attribute_type': 'integer', 'name': 'codigo'},
   {'attribute_type': 'string', 'name': 'comprador'},
   {'attribute_type': 'string', 'name': 'vendedor'},
   {'attribute_type': 'string', 'name': 'descripcion'